# Airline overbooking

## Homework Assigmment 1.6 p.11 Lecture notes

# Exercise 1.6 (Airline Overbooking)

An airline seeks a reservation policy for a flight with $S$ seats that maximizes its expected profit from the flight. Reservation requests arrive hourly according to a Bernoulli process with $p$ being the probability of a reservation request per hour (at most one reservation request will arrive per hour). A passenger with a booked reservation pays the fare $f > 0$ at flight time. If $b \geq 0$ passengers with booked reservations are denied boarding at flight time, they do not pay the fare, and the airline pays them a penalty $c(b)$ (divided among them) where $b \mapsto c(b)$ is increasing with $c(0) = 0$.

Consider the $n$-th hour before flight time $T$. At the beginning of the hour, the airline reviews the number of booked reservations on hand, $r$ say, and decides whether to book (accept) or decline a reservation request arriving during the next hour. Each of the $r$ booked reservations may cancel during the hour, independently of each other, with probability $q$.

For this reason, the airline is considering the possibility of overbooking the flight to compensate for cancellations. Let $V^*_n(r)$ be the maximum expected future profit when there are $r$ booked reservations at the beginning of the hour, before the accept/decline decision has been taken, and reservation requests and cancellations during the hour have occurred. Let $W^*_n(r)$ be the maximum expected future profit when there are $r$ booked reservations after booking or declining a reservation request, but before cancellations. The aim is to determine an optimal reservation policy for any value of the number of booked reservations at the beginning of each hour till the flight time $T$.

### a) Markov Decision Model

Formulate the problem as a Markov decision model, by determining the state space, action spaces, rewards, terminal rewards, and the transition probabilities. Formulate the optimality equation from which an optimal reservation policy can be determined.

### b) Optimality of Booking-Limit Policies

Assume, as can be shown, that if $g$ is a quasiconcave function on the integers, then $r \mapsto \mathbb{E}(g(B_r))$ is quasiconcave, where $B_r$ is a sum of independent identically distributed Bernoulli random variables. We recall that $g$ is quasiconcave on the (positive) integers when there exists a number $a$ such that $g$ is increasing on $[0, a]$ and decreasing on $[a, \infty]$.

Use this result to show the following facts. First, show that $r \mapsto W^*_n(r)$ is quasiconcave.

Let $b_n = \arg\max_r W^*_n(r)$. Call $b_n$ the booking limit. Then show that $r \mapsto V^*_n(r)$ is quasiconcave with maximum $b_n$. Finally, show that it is optimal to accept a reservation if and only if $r < b_n$, with $r$ the number of booked reservations on hand at the beginning of the hour (before a decision has been taken).

### c) Solving the Problem

Solve the problem when the parameters are as follows:

-   $T = 30$
-   $c(b) = f \cdot b$
-   $S = 10$
-   $f = € 300$
-   $p = 0.2$ and $0.3$
-   $q = 0.05$ and $0.10$
-   $r \leq 20$ (so there is an upper bound on the total number of reservations).

Make graphs of the different combinations. In each case, estimate the booking limit ten hours before flight time from your graphs. Discuss whether your graphs confirm the claim in (b) that $r \mapsto V^*_n(r)$ is quasiconcave.

What conjectures do the graphs suggest about the optimal reservation policy and/or maximum expected reward and their variation with the various data elements? You will lose points on your conjectures only if your graphs are inconsistent with or do not support your conjectures, or if you don’t make enough interesting conjectures. The idea here is to brainstorm intelligently.

In [5]:
from scipy.stats import binom

T = 30
f = 300
S = 10
p = 0.2
q = 0.1
r_max = 20

def c(b):
    return 300*b

def solve_mdp(p, q):

    V_opt = {}
    policy_opt = {}

    for n in range(T+1):

        V_opt[n] = {}
        policy_opt[n] = {}

        if n==0:
            for r in  range(T+1):
                if r <= S:
                    V_opt[n][r] = f*r
                else:
                    V_opt[n][r] = f*S - c(r - S)

        else:
            for r in range(T+1 - n):
            
                cancellations = {
                    c: binom.pmf(k=c, n=r, p=q) for c in range(r)
                }
                V_accept = sum([
                    prob_c*(p*V_opt[n-1][r+1-c] + (1-p)*V_opt[n-1][r-c])
                    for c, prob_c in cancellations.items()
                ])
                V_reject = sum([
                    prob_c*V_opt[n-1][r-c]
                    for c, prob_c in cancellations.items()
                ])

                if r >= r_max:
                    policy_opt[n][r] = "reject"
                    V_opt[n][r] = V_reject
                else:
                    if V_accept >= V_reject:
                        policy_opt[n][r] = "accept"
                        V_opt[n][r] = V_accept
                    else:
                        policy_opt[n][r] = "reject"
                        V_opt[n][r] = V_reject
    
    return policy_opt, V_opt

def print_policy(policy_opt, V_opt):
    for n, policy_n in policy_opt.items():
        print(f"Epochs before flight: {n}")
        for r in policy_n:
            print(f"When number of bookings is {r} then policy: {policy_n[r]}, with value {V_opt[n][r]:.2f}")

In [6]:
p = 0.2
q = 0.05
policy_opt, V_opt = solve_mdp(p, q)
print_policy(policy_opt, V_opt)

Epochs before flight: 0
Epochs before flight: 1
When number of bookings is 0 then policy: accept, with value 0.00
When number of bookings is 1 then policy: accept, with value 342.00
When number of bookings is 2 then policy: accept, with value 629.85
When number of bookings is 3 then policy: accept, with value 914.99
When number of bookings is 4 then policy: accept, with value 1200.00
When number of bookings is 5 then policy: accept, with value 1485.00
When number of bookings is 6 then policy: accept, with value 1770.00
When number of bookings is 7 then policy: accept, with value 2055.00
When number of bookings is 8 then policy: accept, with value 2340.00
When number of bookings is 9 then policy: accept, with value 2625.00
When number of bookings is 10 then policy: reject, with value 2850.00
When number of bookings is 11 then policy: reject, with value 2793.72
When number of bookings is 12 then policy: reject, with value 2566.80
When number of bookings is 13 then policy: reject, with va

In [5]:
p = 0.2
q = 0.1
policy_opt, V_opt = solve_mdp(p, q)
print_polict(policy_opt)

Epochs before flight: 0
Epochs before flight: 1
   When number of bookings is 0 then policy: accept
   When number of bookings is 1 then policy: accept
   When number of bookings is 2 then policy: accept
   When number of bookings is 3 then policy: accept
   When number of bookings is 4 then policy: accept
   When number of bookings is 5 then policy: accept
   When number of bookings is 6 then policy: accept
   When number of bookings is 7 then policy: accept
   When number of bookings is 8 then policy: accept
   When number of bookings is 9 then policy: accept
   When number of bookings is 10 then policy: accept
   When number of bookings is 11 then policy: reject
   When number of bookings is 12 then policy: reject
   When number of bookings is 13 then policy: reject
   When number of bookings is 14 then policy: reject
   When number of bookings is 15 then policy: reject
   When number of bookings is 16 then policy: reject
   When number of bookings is 17 then policy: reject
   When 

In [6]:
p = 0.3
q = 0.05
policy_opt, V_opt = solve_mdp(p, q)
print_polict(policy_opt)

Epochs before flight: 0
Epochs before flight: 1
   When number of bookings is 0 then policy: accept
   When number of bookings is 1 then policy: accept
   When number of bookings is 2 then policy: accept
   When number of bookings is 3 then policy: accept
   When number of bookings is 4 then policy: accept
   When number of bookings is 5 then policy: accept
   When number of bookings is 6 then policy: accept
   When number of bookings is 7 then policy: accept
   When number of bookings is 8 then policy: accept
   When number of bookings is 9 then policy: accept
   When number of bookings is 10 then policy: reject
   When number of bookings is 11 then policy: reject
   When number of bookings is 12 then policy: reject
   When number of bookings is 13 then policy: reject
   When number of bookings is 14 then policy: reject
   When number of bookings is 15 then policy: reject
   When number of bookings is 16 then policy: reject
   When number of bookings is 17 then policy: reject
   When 

In [7]:
p = 0.3
q = 0.1
policy_opt, V_opt = solve_mdp(p, q)
print_polict(policy_opt)

Epochs before flight: 0
Epochs before flight: 1
   When number of bookings is 0 then policy: accept
   When number of bookings is 1 then policy: accept
   When number of bookings is 2 then policy: accept
   When number of bookings is 3 then policy: accept
   When number of bookings is 4 then policy: accept
   When number of bookings is 5 then policy: accept
   When number of bookings is 6 then policy: accept
   When number of bookings is 7 then policy: accept
   When number of bookings is 8 then policy: accept
   When number of bookings is 9 then policy: accept
   When number of bookings is 10 then policy: accept
   When number of bookings is 11 then policy: reject
   When number of bookings is 12 then policy: reject
   When number of bookings is 13 then policy: reject
   When number of bookings is 14 then policy: reject
   When number of bookings is 15 then policy: reject
   When number of bookings is 16 then policy: reject
   When number of bookings is 17 then policy: reject
   When 